In [1]:
!pip install thop

In [2]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [4]:
import sys, os
project_root = '/content/drive/MyDrive/regularization-ml'

sys.path.append(project_root)
%cd {project_root}

device = "cuda" if torch.cuda.is_available() else "cpu"

/content/drive/MyDrive/regularization-ml


In [5]:
import torch
from PIL import Image

from src.model import MiniCNN, ConvBlock, FCBlock
from src.train import trainModel
from src.data import CustomDataset, load_cifar_10_data, check_data_loading, Loader, class_to_idx
from src.visualizations import plotFmaps_and_activationHist, plotCurves
from src.utils import EarlyStopping, unpickle, loadWeights, readJson, genError, saveHistory, evalModel

In [6]:
# Copy once from Drive
!cp /content/drive/MyDrive/regularization-ml/data/cifar-10-python.tar.gz /content/

# Extract locally
!mkdir /content/dataset/
!tar -xvzf /content/cifar-10-python.tar.gz -C /content/dataset/

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [7]:
# for the second model, we'll experiment on the effects of cutout at a value of 0.3 in the fc layer
base_conv_layers = [
    ConvBlock(3, 64, pool=False),
    ConvBlock(64, 64),
    ConvBlock(64, 128, pool=False),
    ConvBlock(128, 128)
]

base_fc_layers = [
    FCBlock(128, 64, True, 0.3),
    torch.nn.Linear(64, 10)
]

drop_3_model = MiniCNN(base_conv_layers, base_fc_layers)

In [8]:
%cd /content

/content


In [9]:
train_data, train_labels, val_data, val_labels, test_data, test_labels = load_cifar_10_data()

In [10]:
train_loader, val_loader, test_loader = Loader(train_data, train_labels, val_data, val_labels, test_data, test_labels)

Starting Data Loading...
✅ CUDA available: Tesla T4
   Memory: 15095 MB
📁 Loading datasets...
✅ Datasets loaded successfully
Training samples: 40000
Validation samples: 10000
Batch size: 64
🔍 Testing data loading...
✅ Train batch shape: torch.Size([64, 3, 32, 32]), Labels: torch.Size([64])
   Input range: [-1.989, 2.126]
   Label range: [0, 9]
✅ Val batch shape: torch.Size([64, 3, 32, 32]), Labels: torch.Size([64])


In [ ]:
# we're using the same dataset, and model architecture so let's train
history_drop_3 = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": [], "last_epoch": 0, "LR": []}
dropoutfc0 = "dropoutfc0.3"
drop_03_trained = trainModel(drop_3_model, history_drop_3, train_loader, val_loader, dropoutfc0)
saveHistory(history_drop_3, dropoutfc0)

Using device: cuda
--------------------------------------------------
Epoch 1/70


Current learning rate: 0.001
Train Loss: 1.9562, Train Acc: 0.2489
Val Loss: 1.6305, Val Acc: 0.3710
Best Val Acc: 0.0000 | LR: 0.001000
Time: 38.6s
--------------------------------------------------


--------------------------------------------------
Epoch 2/70


Current learning rate: 0.001
💾 Saved best model!
Train Loss: 1.6161, Train Acc: 0.3861
Val Loss: 1.4341, Val Acc: 0.4589
Best Val Acc: 0.0000 | LR: 0.001000
Time: 36.1s
--------------------------------------------------


--------------------------------------------------
Epoch 3/70


Current learning rate: 0.001
💾 Saved best model!
Train Loss: 1.4318, Train Acc: 0.4752
Val Loss: 1.3861, Val Acc: 0.4743
Best Val Acc: 0.0000 | LR: 0.001000
Time: 36.5s
--------------------------------------------------


--------------------------------------------------
Epoch 4/70


Current learning rate: 0.001
💾 Saved best model!
Train Loss: 1.3147, Train Acc: 0.5203
Val Loss: 1.1964, Val Acc: 0.5589
Best Val Acc: 0.0000 | LR: 0.001000
Time: 35.3s
--------------------------------------------------


--------------------------------------------------
Epoch 5/70


...

Current learning rate: 0.001
💾 Saved best model!
Train Loss: 0.5776, Train Acc: 0.8005
Val Loss: 0.5959, Val Acc: 0.7963
Best Val Acc: 0.0000 | LR: 0.001000
Time: 35.6s
--------------------------------------------------


--------------------------------------------------
Epoch 30/70


Current learning rate: 0.001
Train Loss: 0.5681, Train Acc: 0.8055
Val Loss: 0.6270, Val Acc: 0.7879
Best Val Acc: 0.0000 | LR: 0.001000
Time: 37.0s
--------------------------------------------------


--------------------------------------------------
Epoch 31/70


Current learning rate: 0.001
💾 Saved best model!
Train Loss: 0.5585, Train Acc: 0.8080
Val Loss: 0.5933, Val Acc: 0.7970
Best Val Acc: 0.0000 | LR: 0.001000
Time: 35.3s
--------------------------------------------------


--------------------------------------------------
Epoch 32/70


Current learning rate: 0.0005
Train Loss: 0.5514, Train Acc: 0.8122
Val Loss: 0.6368, Val Acc: 0.7830
Best Val Acc: 0.0000 | LR: 0.000500
Time: 36.9s
--------------------------------------------------


--------------------------------------------------
Epoch 33/70


Training:  85%|████████▍ | 531/625 [00:28<00:04, 21.80it/s, loss=0.5601, batch=530/625]

In [ ]:
plotFmaps_and_activationHist(drop_3_model, "dropoutfc0.3", "model_best", val_loader, 1)

In [ ]:
plotFmaps_and_activationHist(drop_3_model, "dropoutfc0.3", "model_best", val_loader, 3)

In [ ]:
baseline_data = readJson("baseline")
print(baseline_data.keys())

In [ ]:
train_losses = baseline_data["train_loss"]
val_losses = baseline_data["val_loss"]
train_accs = baseline_data["train_acc"]
val_accs = baseline_data["val_acc"]
plotCurves(train_losses, val_losses, train_accs, val_accs, "dropoutfc0.3")

In [ ]:
evalModel(drop_3_model, "baseline", "model_best", test_loader)